#### REF
* https://github.com/dmlc/gluon-nlp/blob/master/docs/api/notes/data_api.rst

## Data Loading

In [1]:
import os
import pandas as pd
import numpy as np
import collections
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score, auc
from mxnet import gluon


import time, re
import multiprocessing as mp
import itertools
from tqdm import tqdm, tqdm_notebook
import mxnet as mx
import spacy
os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'

In [2]:
MAX_SENTENCE_LENGTH = 20
MAX_VOCAB = 10000

### Preprocessing using Spacy

In [3]:
nlp = spacy.load("en")

word_freq = collections.Counter()
max_len = 0
num_rec = 0
print('Count words and build vocab...')
with open('../data/umich-sentiment-train.txt', 'rb') as f:
    for line in f:
        _lab, _sen = line.decode('utf8').strip().split('\t')
        words = [token.lemma_ for token in nlp(_sen) if token.is_alpha] # Stop word제거 안한 상태 
        # 제거를 위해 [token.lemma_ for token in doc if token.is_alpha and not token.is_stop]
        if len(words) > max_len:
            max_len = len(words)
        for word in words:
            word_freq[word] += 1
        num_rec += 1

# most_common output -> list
word2idx = {x[0]: i+2 for i, x in enumerate(word_freq.most_common(MAX_VOCAB - 2))}
word2idx ['PAD'] = 0
word2idx['UNK'] = 1

idx2word= {i:v for v, i in word2idx.items()}
vocab_size = len(word2idx)

print('Prepare data...')
y = []
x = []
origin_txt = []
with open('../data/umich-sentiment-train.txt', 'rb') as f:
    for line in f:
        _label, _sen = line.decode('utf8').strip().split('\t')
        origin_txt.append(_sen)
        y.append(int(_label))
        words = [token.lemma_ for token in nlp(_sen) if token.is_alpha] # Stop word제거 안한 상태
        words = [x for x in words if x != '-PRON-'] # '-PRON-' 제거
        _seq = []
        for word in words:
            if word in word2idx.keys():
                _seq.append(word2idx[word])
            else:
                _seq.append(word2idx['UNK'])
        if len(_seq) < MAX_SENTENCE_LENGTH:
            _seq.extend([0] * ((MAX_SENTENCE_LENGTH) - len(_seq)))
        else:
            _seq = _seq[:MAX_SENTENCE_LENGTH]
        x.append(_seq)

pd.DataFrame(y, columns = ['yn']).reset_index().groupby('yn').count().reset_index()

Count words and build vocab...
Prepare data...


,yn,index
0,0,3091
1,1,3995


In [4]:
## Data process - tr/va split and define iterator

tr_idx = np.random.choice(range(len(x)), int(len(x) * .8))
va_idx = [x for x in range(len(x)) if x not in tr_idx]

tr_x = [x[i] for i in tr_idx]
tr_y = [y[i] for i in tr_idx]
tr_origin = [origin_txt[i] for i in tr_idx]

va_x = [x[i] for i in va_idx]
va_y = [y[i] for i in va_idx]
va_origin = [origin_txt[i] for i in va_idx]

batch_size = 16

learning_rate = .0002
log_interval = 100


train_data = mx.io.NDArrayIter(data=[tr_x, tr_y], batch_size=batch_size, shuffle = False, last_batch_handle = 'discard')
valid_data = mx.io.NDArrayIter(data=[va_x, va_y], batch_size=batch_size, shuffle = False, last_batch_handle = 'discard')

In [5]:
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn, rnn
from mxnet.ndarray.linalg import gemm2
import mxnet as mx
context = mx.gpu()

#### Sentence Representation

In [15]:
class Sentence_Representation(nn.Block):
    def __init__(self, **kwargs):
        super(Sentence_Representation, self).__init__()
        for (k, v) in kwargs.items():
            setattr(self, k, v)
        
        with self.name_scope():
            self.embed = nn.Embedding(self.vocab_size, self.emb_dim)
            self.g_fc1 = nn.Dense(self.hidden_dim, activation='relu', flatten = False)
            self.g_fc2 = nn.Dense(self.hidden_dim, activation='relu', flatten = False)
            
    def forward(self, x):
        embeds = self.embed(x) # batch * time step * embedding
        x_i = embeds.expand_dims(1) # batch * 1 * time step * embedding
        x_i = nd.repeat(x_i,repeats= self.sentence_length, axis=1)
        x_j = embeds.expand_dims(2) # batch * time step * 1 * embedding
        x_j = nd.repeat(x_j,repeats= self.sentence_length, axis=2)
        x_full = nd.concat(x_i,x_j,dim=3) # batch * time step * time step * (2 * embedding)
        # batch* time step * time step 개의 batch를 가진 2*VOCABULARY input을 network에 feed

        _x = self.g_fc1(x_full) # (64 * 40 * 40) * 256: .1GB 추가메모리는 안먹나?
        print('_x shape after g_1 = {}'.format(_x.shape))
        _x = self.g_fc2(_x) # (64 * 40 * 40) * 256: .1GB (reuse)
        print('_x shape after g_2 = {}'.format(_x.shape))
        # sentence_length*sentence_length개의 결과값을 모두 합해서 sentence representation으로 나타냄
        x_g = _x.reshape((-1, self.sentence_length * self.sentence_length, self.hidden_dim))
        sentence_rep = x_g.sum(1) # (64, 256): ignorable
        return sentence_rep

#### Classifier

In [9]:
classifier = nn.Sequential()
classifier.add(nn.Dense(16, activation = 'relu'))
classifier.add(nn.Dense(8, activation = 'relu'))
classifier.add(nn.Dense(1))
classifier.collect_params().initialize(mx.init.Xavier(), ctx = context)

#### Sentiment analysis classifier

In [16]:
class SA_Classifier(nn.Block):
    def __init__(self, sen_rep, classifier, context, **kwargs):
        super(SA_Classifier, self).__init__(**kwargs)
        self.context = context
        with self.name_scope():
            self.sen_rep = sen_rep
            self.classifier = classifier
            
    def forward(self, x):
        # Initial hidden state
        # sentence representation할 때 hidden의 context가 cpu여서 오류 발생. context를 gpu로 전환
        x = self.sen_rep(x)
        x = nd.flatten(x)
        res = self.classifier(x)
        return res       

#### Initiate sentiment classifier

In [17]:
emb_dim = 10 # Emb dim
hidden_dim = 30 # Hidden dim for LSTM

param = {'emb_dim': emb_dim, 'hidden_dim': hidden_dim, 'vocab_size': vocab_size, 'sentence_length': MAX_SENTENCE_LENGTH, 'dropout': .2}
sen_rep = Sentence_Representation(**param)
sen_rep.collect_params().initialize(mx.init.Xavier(), ctx = context)


In [18]:
sa = SA_Classifier(sen_rep, classifier, context)
loss = gluon.loss.SigmoidBCELoss()
trainer = gluon.Trainer(sa.collect_params(), 'adam', {'learning_rate': 1e-3})
sa.hybridize()

/opt/venv/lib/python3.6/site-packages/mxnet/gluon/block.py:521: UserWarning: All children of this Sequential layer 'sequential1_' are HybridBlocks. Consider using HybridSequential for the best performance.
  cld.hybridize(active, **kwargs)


In [19]:
def evaluate(net, dataIterator, context):
    dataIterator.reset()
    loss = gluon.loss.SigmoidBCELoss()
    total_L = 0.0
    total_sample_num = 0
    total_correct_num = 0
    start_log_interval_time = time.time()
    for i, batch in enumerate(dataIterator):
        data =  batch.data[0].as_in_context(context)
        label = batch.data[1].as_in_context(context)
        output = net(data)
        L = loss(output, label)
        pred = (output > 0.5).reshape((-1,))
        total_L += L.sum().asscalar()
        total_sample_num += len(label)
        total_correct_num += (pred == label).sum().asscalar()
        if (i + 1) % log_interval == 0:
            print('[Batch {}/{}] elapsed {:.2f} s'.format(
                i + 1, dataIterator.num_data//dataIterator.batch_size,
                time.time() - start_log_interval_time))
            start_log_interval_time = time.time()
    avg_L = total_L / float(total_sample_num)
    acc = total_correct_num / float(total_sample_num)
    return avg_L, acc

In [20]:
n_epoch = 5
for epoch in tqdm_notebook(range(n_epoch), desc = 'epoch'):
    ## Training
    train_data.reset()
    # Epoch training stats
    start_epoch_time = time.time()
    epoch_L = 0.0
    epoch_sent_num = 0
    epoch_wc = 0
    # Log interval training stats
    start_log_interval_time = time.time()
    log_interval_wc = 0
    log_interval_sent_num = 0
    log_interval_L = 0.0
    
    for i, batch in enumerate(train_data):
        _data = batch.data[0].as_in_context(context)
        _label = batch.data[1].as_in_context(context)
        L = 0
        wc = len(_data)
        log_interval_wc += wc
        epoch_wc += wc
        log_interval_sent_num += _data.shape[1]
        epoch_sent_num += _data.shape[1]
        with autograd.record():
            _out = sa(_data)
            L = L + loss(_out, _label).mean().as_in_context(context)
        L.backward()
        trainer.step(_data.shape[0])
        log_interval_L += L.asscalar()
        epoch_L += L.asscalar()
        if (i + 1) % log_interval == 0:
            tqdm.write('[Epoch {} Batch {}/{}] elapsed {:.2f} s, \
                    avg loss {:.6f}, throughput {:.2f}K wps'.format(
                    epoch, i + 1, train_data.num_data//train_data.batch_size,
                    time.time() - start_log_interval_time,
                    log_interval_L / log_interval_sent_num,
                    log_interval_wc / 1000 / (time.time() - start_log_interval_time)))
            # Clear log interval training stats
            start_log_interval_time = time.time()
            log_interval_wc = 0
            log_interval_sent_num = 0
            log_interval_L = 0
    end_epoch_time = time.time()
    test_avg_L, test_acc = evaluate(sa, valid_data, context)
    tqdm.write('[Epoch {}] train avg loss {:.6f}, valid acc {:.2f}, \
        valid avg loss {:.6f}, throughput {:.2f}K wps'.format(
        epoch, epoch_L / epoch_sent_num,
        test_acc, test_avg_L, epoch_wc / 1000 /
        (end_epoch_time - start_epoch_time)))

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
[Epoch 0] train avg loss 0.007159, valid acc 0.99,         valid avg loss 0.038715, thro

...

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

### Prediction

In [21]:
def get_pred(net, iterator):
    pred_sa = []
    label_sa = []
    va_text = []
    iterator.reset()
    for i, batch in enumerate(iterator):
        if i % 100 == 0:
            print('i = {}'.format(i))
        data =  batch.data[0].as_in_context(context)
        label = batch.data[1].as_in_context(context)
        output = net(data)
        L = loss(output, label)
        pred = (nd.sigmoid(output) > 0.5).reshape((-1,))
        pred_sa.extend(pred.asnumpy())
        label_sa.extend(label.asnumpy())
        va_text.extend([' '.join([idx2word[np.int(x)] for x in y.asnumpy() if idx2word[np.int(x)] is not 'PAD']) for y in data])
    pred_sa_pd = pd.DataFrame(pred_sa, columns  = ['pred_sa'])
    label_pd = pd.DataFrame(label_sa, columns = ['label'])
    text_pd = pd.DataFrame(va_text, columns = ['text'])
    res = pd.concat([text_pd, pred_sa_pd, label_pd], axis = 1)
    return res

## Classification results

In [22]:
result = get_pred(sa, valid_data)

i = 0
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x sha

_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape after g_1 = (16, 20, 20, 30)
_x shape after g_2 = (16, 20, 20, 30)
_x shape aft

In [23]:
result[result.pred_sa != result.label].shape

(30, 3)

In [24]:
result[result.pred_sa != result.label].head(10)

,text,pred_sa,label
454,ok time to update wow have update for a long t...,0.0,1.0
459,finally feel up to make the long ass drive out...,0.0,1.0
460,finally feel up to make the long ass drive out...,0.0,1.0
518,i love the first mission impossible the second...,0.0,1.0
526,hate though because really like mission imposs...,0.0,1.0
530,hate though because really like mission imposs...,0.0,1.0
918,child text fantasy perhaps most obviously be o...,0.0,1.0
946,harry potter and all of magical lore,0.0,1.0
966,harry potter series because of the delicious w...,0.0,1.0
973,do get wrong like harry potter but need a fant...,0.0,1.0


In [25]:
result[result.pred_sa != result.label]['text'].iloc[1]

'finally feel up to make the long ass drive out to the haunt tonight mission impossible iii be good'